In [4]:
import numpy as np
import pandas as pd
import sys
original_path = sys.path.copy()
sys.path.insert(0, '../../nodefeaturing')
from generating_feature import generating_pro_feature, generating_drug_feature, concat_feature
sys.path = original_path

/home/max38744/.conda/envs/moon/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [45]:
file_path = "dti2vec_drugbankid2smiles"  # Specify the file path without the extension

# Read the file as a CSV with custom parameters

df = pd.read_csv(file_path, header=None, delimiter="\t")  # Assuming the file has no header row
print(len(df))
print(df.iloc[[0,1,300,600, 1000]])
df.columns = ["Drug_ID", "Drug"]

9881
            0                                                  1
0     DB00672                 CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl
1     DB00116  C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N...
300   DB00419          CCCCN1C[C@@H]([C@H]([C@@H]([C@H]1CO)O)O)O
600   DB00714       CN1CCC2=C3[C@H]1CC4=C(C3=CC=C2)C(=C(C=C4)O)O
1000  DB00497  CN1CC[C@]23[C@@H]4C(=O)CC[C@]2([C@H]1CC5=C3C(=...


In [93]:
import pickle
import json
temp = []

    

# JSON 파일 경로 설정
json_file = 'node2id.json'

# JSON 파일 읽기
with open(json_file) as f:
    data = json.load(f)
    
for idx, item in enumerate(tmp["Drug_ID"].copy()) :

    for jdx, d_name in enumerate(data.keys()) :
        if d_name == item :
            temp.append(data[d_name])
            break
        elif jdx +1 == len(data)  :
            temp.append(np.NaN)


In [97]:
# Select unique rows based on the 'Drug' column
unique_drugs_df = df.drop_duplicates(subset=['Drug']).dropna()

unique_drugs_df = generating_drug_feature(unique_drugs_df)
# Print the resulting DataFrame
# print(len(unique_drugs_df))
# print(unique_drugs_df)
tmp = unique_drugs_df[['Drug_ID', 'Drug','Morgan_Features']]
# print(tmp.head())

drug_id_array = tmp['Drug_ID'].values
drug_array = tmp['Drug'].values
drug_feat = tmp['Morgan_Features']
drug_enco = temp

drug_feat = {'Drug_ID': drug_id_array, 'Drug': drug_array, 'Morgan_Features': drug_feat, "Drug_enco" : drug_enco}



import pickle


with open("VEC_drug_feats.pkl", 'wb') as f:
    pickle.dump(drug_feat, f)

print(f"Dictionary saved as pickle file")

Dictionary saved as pickle file


In [100]:

with open('VEC_drug_feats.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
x

{'Drug_ID': array(['DB00672', 'DB00116', 'DB06663', ..., 'DB00610', 'DB08903',
        'DB00474'], dtype=object),
 'Drug': array(['CCCNC(=O)NS(=O)(=O)C1=CC=C(C=C1)Cl',
        'C1C(NC2=C(N1)N=C(NC2=O)N)CNC3=CC=C(C=C3)C(=O)N[C@@H](CCC(=O)O)C(=O)O',
        'C1[C@H](CN2[C@@H]1C(=O)N[C@H](C(=O)N[C@@H](C(=O)N[C@H](C(=O)N[C@H](C(=O)N[C@H](C2=O)CC3=CC=CC=C3)CC4=CC=C(C=C4)OCC5=CC=CC=C5)CCCCN)CC6=CNC7=CC=CC=C76)C8=CC=CC=C8)OC(=O)NCCN',
        ..., 'C[C@@H]([C@@H](C1=CC(=CC=C1)O)O)N',
        'CN(C)CC[C@@](C1=CC=CC2=CC=CC=C21)([C@H](C3=CC=CC=C3)C4=C(N=C5C=CC(=CC5=C4)Br)OC)O',
        'CCC#CC(C)C1(C(=O)NC(=O)N(C1=O)C)CC=C'], dtype=object),
 'Morgan_Features': 0       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
 1       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
 2       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
 4       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
 6       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                               ...             

In [101]:
with open('entity_drug.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df = df.T
df

,0
Gene::801,1710
Gene::7428,1711
Gene::5987,1712
Gene::9412,1713
Gene::5747,1714
...,...
Gene::255349,34119
Gene::387104,34120
Gene::347541,34121
Gene::4114,34122


In [106]:
df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]

In [108]:
df = pd.read_csv("origin_proteins.csv").drop("Unnamed: 0", axis = 1)
df.columns = ["Target_ID", "Target"]
# Select unique rows based on the 'Drug' column
unique_targets_df = df.drop_duplicates(subset=['Target'])
print(len(unique_targets_df))

pro_df = generating_pro_feature(unique_targets_df)
# Print the resulting DataFrame
# print(len(pro_df))
# print(pro_df)
tmp = pro_df[['Target_ID', 'Target','ProtBERT_Features']]
print(tmp.head())

target_id_array = tmp['Target_ID'].values
target_array = tmp['Target'].values
target_feat = tmp['ProtBERT_Features']

target_feat = {'Target_ID': target_id_array, 'Target': target_array, 'ProtBERT_Features': target_feat}

print(len(target_feat['ProtBERT_Features']))
print(type(target_feat['ProtBERT_Features']))
print(target_feat)

import pickle


with open("VEC_target_feats_tmp.pkl", 'wb') as f:
    pickle.dump(target_feat, f)

print(f"Dictionary saved as pickle file")


1407


Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Defa

  Target_ID                                             Target  \
0    A5X5Y0  MLAFILSRATPRPALGPLSYREHRVALLHLTHSMSTTGRGVTFTIN...   
1    Q03013  MSMTLGYWDIRGLAHAIRLLLEYTDSSYEEKKYTMGDAPDYDRSQW...   
2    Q04844  MARAPLGVLLLLGLLGRGVGKNEELRLYHHLFNNYDPGSRPVREPE...   
3    Q04828  MDSKYQCVKLNDGHFMPVLGFGTYAPAEVPKSKALEATKLAIEAGF...   
4    Q04771  MVDGVMILPVLIMIALPSPSMEDEKPKVNPKLYMCVCEGLSCGNED...   

                                   ProtBERT_Features  
0  [0.030560227, 0.02434674, 0.13633652, 0.032938...  
1  [0.030560227, 0.02434674, 0.13633652, 0.032938...  
2  [0.030560227, 0.02434674, 0.13633652, 0.032938...  
3  [0.030560227, 0.02434674, 0.13633652, 0.032938...  
4  [0.030560227, 0.02434674, 0.13633652, 0.032938...  
1407
<class 'pandas.core.series.Series'>
{'Target_ID': array(['A5X5Y0', 'Q03013', 'Q04844', ..., 'P18031', 'P17948', 'Q9Y6R1'],
      dtype=object), 'Target': array(['MLAFILSRATPRPALGPLSYREHRVALLHLTHSMSTTGRGVTFTINCSGFGQHGADPTALNSVFNRKPFRPVTNISVPTQVNISFAMSAILDVNEQLHLLSSFLWL

In [109]:
with open('entity_drug.json', 'r') as f:
    data = json.load(f)

df = pd.json_normalize(data)

df = df.T
df

,0
Gene::801,1710
Gene::7428,1711
Gene::5987,1712
Gene::9412,1713
Gene::5747,1714
...,...
Gene::255349,34119
Gene::387104,34120
Gene::347541,34121
Gene::4114,34122


In [117]:
a

,UniProtKB Gene Name ID,Gene name,NCBI gene (formerly Entrezgene) ID
0,A0A023T6R1,MAGOHB,Gene::55110
1,A0A023T787,RBM8A,Gene::9939
2,A0A024QY88,NRG1,Gene::3084
3,A0A024QYR8,TM9SF2,Gene::9375
4,A0A024QYT5,SERPINE1,Gene::5054
...,...,...,...
73808,X6RLV5,DDX5,Gene::1655
73809,X6RLX0,ERC1,Gene::23085
73810,X6RLY7,CACNA2D4,Gene::93589
73811,X6RM00,ERC1,Gene::23085


In [123]:
import pandas as pd
a = pd.read_csv("convert_table", delimiter= "\t")

new = []
new_emb = []
for x_idx, pro in enumerate(target_feat['Target_ID']) :
    for a_idx, name in enumerate(a["UniProtKB Gene Name ID"]) : 
        if pro == name :
            new.append(a['NCBI gene (formerly Entrezgene) ID'].iloc[a_idx])
            new_emb.append(target_feat["ProtBERT_Features"][x_idx])
            break


In [124]:
new_x = {}
new_x["Target_ID"] = new
new_x["ProtBERT_Features"]= new_emb
new_x = pd.DataFrame(new_x)
print(new_x)

         Target_ID                                  ProtBERT_Features
0     Gene::285242  [0.030560227, 0.02434674, 0.13633652, 0.032938...
1       Gene::2948  [0.030560227, 0.02434674, 0.13633652, 0.032938...
2       Gene::1145  [0.030560227, 0.02434674, 0.13633652, 0.032938...
3       Gene::1645  [0.030560227, 0.02434674, 0.13633652, 0.032938...
4         Gene::90  [0.030560227, 0.02434674, 0.13633652, 0.032938...
...            ...                                                ...
1380    Gene::6715  [0.030560227, 0.02434674, 0.13633652, 0.032938...
1381     Gene::151  [0.030560227, 0.02434674, 0.13633652, 0.032938...
1382    Gene::5770  [0.030560227, 0.02434674, 0.13633652, 0.032938...
1383    Gene::2321  [0.030560227, 0.02434674, 0.13633652, 0.032938...
1384    Gene::8671  [0.030560227, 0.02434674, 0.13633652, 0.032938...

[1385 rows x 2 columns]


In [126]:
import json

with open('entity_drug.json', 'r') as f:
    data = json.load(f)
enco = []
for i in new :
    if i in data.keys() :
        enco.append(data[i])
    else : 
        enco.append(np.NaN)
print(enco)
result["Gene_enco"] = enco
result = result.sort_values(by = "Gene_enco").reset_index(drop=True)
result["mapping_arr"] = range(0,len(result))

[21172, 5985, 16479, 15880, 3726, 10246, 10134, 8078, 7017, 5712, 12274, 2093, 8031, 7350, 4559, 2987, 13915, 8426, 13979, 2305, 18835, 8254, 15695, 15106, 7619, 6471, 13830, 3344, 7220, 9205, 15274, 6545, 15779, 5894, 13102, 5052, 11962, 6364, 13198, 3956, 11158, 14254, 15936, 14006, 3849, 3697, 3407, 15456, 18790, 6899, 3941, 2759, 1763, 4538, 4777, 6942, 2532, 3200, 2731, 6587, 8406, 10569, 2671, 2760, 16488, 10451, 4163, 7139, 14132, 8012, 15393, 10746, 12902, 6192, 2182, 13734, 7690, 2570, 8516, 2298, 3078, 2029, 9233, 15363, 2469, 8437, 4009, 3326, 3429, 2814, 17651, 7861, 16273, 12160, 3555, 18923, 15549, nan, 4946, 11143, 7105, 7406, 13261, 9597, 7895, 13165, 3117, 4034, 2033, 13199, 12806, 11510, 17826, 15211, 9856, 3255, 13086, 5568, 12122, 6381, 3965, 11060, 15094, 16398, 4116, 16677, 9426, 18856, 16205, 4356, 16524, 7344, 16724, 13492, 6632, 4231, 18812, 2454, 4994, 10238, 7454, 9936, 2758, 3163, 5406, 10991, 9823, 14848, 9158, 4904, 2983, 3253, 19632, 3772, 4859, 8844, 404

,Target_ID,ProtBERT_Features,Gene_enco,mapping_arr
0,Gene::3725,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",1716.0,0
1,Gene::10014,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",1718.0,1
2,Gene::2099,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",1720.0,2
3,Gene::960,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",1726.0,3
4,Gene::1017,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",1737.0,4
...,...,...,...,...
1380,Gene::404037,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",21260.0,1380
1381,Gene::200909,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",22598.0,1381
1382,Gene::107080644,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",NaN,1382
1383,Gene::,"[0.030560227, 0.02434674, 0.13633652, 0.032938...",NaN,1383


In [127]:
result
davis_pfeat = {"Target_ID" : result["Target_ID"].values, "ProtBERT_Features" : result["ProtBERT_Features"], "Gene_enco" : result["Gene_enco"].values, "map_arr" : result["mapping_arr"].values}
davis_pfeat

{'Target_ID': array(['Gene::3725', 'Gene::10014', 'Gene::2099', ..., 'Gene::107080644',
        'Gene::', 'Gene::104909134'], dtype=object),
 'ProtBERT_Features': 0       [0.030560227, 0.02434674, 0.13633652, 0.032938...
 1       [0.030560227, 0.02434674, 0.13633652, 0.032938...
 2       [0.030560227, 0.02434674, 0.13633652, 0.032938...
 3       [0.030560227, 0.02434674, 0.13633652, 0.032938...
 4       [0.030560227, 0.02434674, 0.13633652, 0.032938...
                               ...                        
 1380    [0.030560227, 0.02434674, 0.13633652, 0.032938...
 1381    [0.030560227, 0.02434674, 0.13633652, 0.032938...
 1382    [0.030560227, 0.02434674, 0.13633652, 0.032938...
 1383    [0.030560227, 0.02434674, 0.13633652, 0.032938...
 1384    [0.030560227, 0.02434674, 0.13633652, 0.032938...
 Name: ProtBERT_Features, Length: 1385, dtype: object,
 'Gene_enco': array([1716., 1718., 1720., ...,   nan,   nan,   nan]),
 'map_arr': array([   0,    1,    2, ..., 1382, 1383, 1384])}

In [136]:
import pickle
with open("VEC_target_feats_tmp.pkl", 'wb') as f:
    pickle.dump(davis_pfeat, f)

print(f"Dictionary saved as pickle file")

Dictionary saved as pickle file


In [99]:

with open('DAVIS_target_feats_tmp.pkl', 'rb') as f:
    x = pickle.load(f, encoding='utf-8')
print(x.keys())
print(type((x['ProtBERT_Features'])))
print(x['ProtBERT_Features'])

dict_keys(['Target_ID', 'ProtBERT_Features', 'Gene_enco', 'map_arr'])
<class 'pandas.core.series.Series'>
0      [0.030560208, 0.024346584, 0.13633646, 0.03293...
1      [0.030560208, 0.024346584, 0.13633646, 0.03293...
2      [0.030560208, 0.024346584, 0.13633646, 0.03293...
3      [0.030560208, 0.024346584, 0.13633646, 0.03293...
4      [0.030560208, 0.024346584, 0.13633646, 0.03293...
                             ...                        
252    [0.030560208, 0.024346584, 0.13633646, 0.03293...
253    [0.030560208, 0.024346584, 0.13633646, 0.03293...
254    [0.030560208, 0.024346584, 0.13633646, 0.03293...
255    [0.030560208, 0.024346584, 0.13633646, 0.03293...
256    [0.030560208, 0.024346584, 0.13633646, 0.03293...
Name: ProtBERT_Features, Length: 257, dtype: object
